In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: pho
"""
import sys
from threading import Thread
from ipygany import PolyMesh, Scene, IsoColor, WarpByScalar
import pyvista as pv
import pyvistaqt as pvqt

# pv.set_jupyter_backend('pythreejs')
# pv.set_jupyter_backend('ipygany') # ipygany fails " Failed to use notebook backend: "
# pv.set_jupyter_backend('panel') # Doesn't work either, produces no output
import numpy as np
import h5py
import hdf5storage # conda install hdf5storage
from pathlib import Path
import bqplot.scales
import seaborn as sns
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
# import mplcursors
from scipy.ndimage.filters import gaussian_filter
import ipywidgets as widgets
# Imports PIL module
import PIL # For applying the binary mask as a image texture to a mesh
# import opencv
from IPython.display import display # For manual display customization

In [2]:
mat_import_file = 'data/RoyMaze1/positionAnalysis.mat'
print('Loading matlab import file: {}...'.format(mat_import_file))
data = hdf5storage.loadmat(mat_import_file, appendmat=False)
# np.shape(data)
print('done.')

Loading matlab import file: data/RoyMaze1/positionAnalysis.mat...
done.


In [3]:
# data.keys()
t = np.squeeze(data['positionalAnalysis']['track_position']['t'])
x = np.squeeze(data['positionalAnalysis']['track_position']['x'])
y = np.squeeze(data['positionalAnalysis']['track_position']['y'])
speeds = np.squeeze(data['positionalAnalysis']['track_position']['speeds'])
dt = np.squeeze(data['positionalAnalysis']['displacement']['dt'])
dx = np.squeeze(data['positionalAnalysis']['displacement']['dx'])
dy = np.squeeze(data['positionalAnalysis']['displacement']['dy'])

print('shapes - t: {}, x: {}, y: {}'.format(np.shape(t), np.shape(x), np.shape(y))) 

extrema_min, extrema_max = np.nanmin(x), np.nanmax(x)
print('for x: extrema_min: {}, extrema_max: {}'.format(extrema_min, extrema_max))
extrema_min, extrema_max = np.nanmin(y), np.nanmax(y)
print('for y: extrema_min: {}, extrema_max: {}'.format(extrema_min, extrema_max))
 

shapes - t: (324574,), x: (324574,), y: (324574,)
for x: extrema_min: 74.29166666666666, extrema_max: 649.3020833333334
for y: extrema_min: 160.54166666666666, extrema_max: 263.0


In [4]:
should_show_2d_plots = False

# Create a representation of the general maze layout by binning the data and then building a mask from all the points the animal ever traversed in the maze
# Output: negligably_traversed_maze_regions - an 80x80 binary mask that specifies which maze regions are never traversed.
def gen_2d_histrogram(x, y, sigma, bins=80):
    heatmap, xedges, yedges = np.histogram2d(x, y, bins=bins, density=False)
    heatmap = gaussian_filter(heatmap, sigma=sigma)
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    return heatmap.T, extent, xedges, yedges
 
# For the 1.6m track, we want bins of ~2cm -> 80 bins
num_position_bins = 80
# position_bins = np.linspace(0.0, 1.6, num_position_bins)
# digitized_x = np.digitize(x, position_bins)
# digitized_y = np.digitize(y, position_bins)

smoothing_sigma = 2
img, extent, xedges, yedges = gen_2d_histrogram(x, y, smoothing_sigma)

## Get the histogram x that corresponds to each x, etc.
digitized_x = np.digitize(x, xedges)-1
digitized_y = np.digitize(y, yedges)-1

get_point_heatmap_value = lambda i, : img.T[(digitized_x[i]-1), (digitized_y[i]-1)]
point_heatmap_value = np.array(list(map(get_point_heatmap_value, range(0, len(digitized_x)))))


# Convert the values into a actual color vectors
cmap = cm.jet
norm = Normalize(vmin=np.min(point_heatmap_value), vmax=np.max(point_heatmap_value))
point_colors = cmap(norm(point_heatmap_value))
print('point_heatmap_value: {}'.format(point_heatmap_value))

# print('img: {}'.format(np.shape(img)))
# img_mask = np.nonzero(img)
negligably_traversed_maze_regions = np.where(img < 25)
# print('negligably_traversed_maze_regions: {}'.format(np.shape(negligably_traversed_maze_regions)))
img_mask = img
img_mask[negligably_traversed_maze_regions] = np.nan
boosted_img = pow(img, 2)
# print('np.shape(img): {}, img: {}'.format(np.shape(img), img))

if should_show_2d_plots:
    fig, ax = plt.subplots(figsize=(15,6))
    plt.imshow(img, extent=extent, origin='lower', cmap=cm.jet)
    plt.colorbar()
    plt.title('Non-negligably Traversed Maze Regions')
    # xx, yy = np.meshgrid(position_bins, position_bins)
    # # xx, yy = np.meshgrid(digitized_x, digitized_y)
    # plt.imshow(z_values, cmap='hot', interpolation='nearest')
    # plt.hexbin(x, y, C=z_values, gridsize=gridsize, cmap=CM.jet, bins=None)
    # ax = sns.heatmap(z_values, linewidth=0.5)
    sns.jointplot(x=x, y=y, kind='hex')
    plt.show()

point_heatmap_value: [207.19981836 207.19981836 207.19981836 ... 181.29856709 181.29856709
 181.29856709]


In [5]:
# point_cloud = np.random.random((100, 3))
# z = point_heatmap_value
z = np.zeros_like(x)
point_cloud = np.vstack((x, y, z)).T
pdata = pv.PolyData(point_cloud)
pdata['orig_sphere'] = np.arange(np.shape(point_cloud)[0])

# create many spheres from the point cloud
sphere = pv.Sphere(radius=0.2, phi_resolution=3, theta_resolution=3)
pc = pdata.glyph(scale=False, geom=sphere)

In [ ]:
# pc.plot(cmap='Reds')
# pc.plot()

p = pv.Plotter()
p.background_color = 'black'
p.add_points(pdata)
p.add_bounding_box()
p.show_grid()
p.show()


In [ ]:
p = pvqt.BackgroundPlotter() # Use just like you would a pv.Plotter() instance
p.background_color = 'black'
p.add_points(pdata)
p.add_spline_widget(tube)
p.add_bounding_box()
p.show_grid()
p.show()



In [ ]:
# pc.plot(cmap='Reds', notebook=True, jupyter_backend='ipygany') # ipygany fails " Failed to use notebook backend: "

In [ ]:
pc.plot(cmap='Reds', notebook=True, jupyter_backend='panel')

In [7]:
## Plot the raw position points

# pvmesh = examples.download_topo_global()
pvmesh = pv.examples.download_st_helens()

ugrid = pvmesh.cast_to_unstructured_grid()

# Turn the PyVista mesh into a PolyMesh
mesh = PolyMesh.from_vtk(ugrid)

colored_mesh = IsoColor(mesh, min=-10421.0, max=6527.0)
warped_mesh = WarpByScalar(colored_mesh, input='altitude', factor=0.5e-5)

Scene([warped_mesh])

AttributeError: module 'pyvista' has no attribute 'examples'

In [ ]:
# Projected Maze 2D Outline:
maze_outline_x = x
maze_outline_y = y


In [14]:
# Split the position data into equal sized chunks to be displayed at a single time. These will look like portions of the trajectory and be used to animate. # Chunk the data to create the animation.
# curr_view_window_length = 150 # View 5 seconds at a time (30fps)
curr_view_window_length = 30 # View 5 seconds at a time (30fps)
# The original length 324574 / 30 = 10819

trimmed_elements = np.remainder(np.size(x), curr_view_window_length) # Compute the number of elements that need to be droppped to be able to evently divide the original arrays into evenly sized chunks of length `curr_view_window_length`
# e.g. np.remainder(324574, 150) = 124

# drop 124 extra elements that make it no wrap evenly
trimmed_length = np.size(x) - trimmed_elements
# e.g. trimmed_length = 324574 - 124 # 324574 - 124
other_reshaped_dimension = np.floor_divide(np.size(x), curr_view_window_length) # e.g. 2163

t_fixedSegements = t[0:trimmed_length].reshape(other_reshaped_dimension, curr_view_window_length)
x_fixedSegements = x[0:trimmed_length].reshape(other_reshaped_dimension, curr_view_window_length)
y_fixedSegements = y[0:trimmed_length].reshape(other_reshaped_dimension, curr_view_window_length)

speeds_fixedSegements = speeds[0:trimmed_length].reshape(other_reshaped_dimension, curr_view_window_length)
dt_fixedSegements = dt[0:trimmed_length].reshape(other_reshaped_dimension, curr_view_window_length)
dx_fixedSegements = dx[0:trimmed_length].reshape(other_reshaped_dimension, curr_view_window_length)
dy_fixedSegements = dy[0:trimmed_length].reshape(other_reshaped_dimension, curr_view_window_length)

print('shapes - t_fixedSegements: {}, x_fixedSegements: {}, y_fixedSegements: {}'.format(np.shape(t_fixedSegements), np.shape(x_fixedSegements), np.shape(y_fixedSegements)))
z_fixedSegements = np.zeros_like(x_fixedSegements)


shapes - t_fixedSegements: (10819, 30), x_fixedSegements: (10819, 30), y_fixedSegements: (10819, 30)


In [31]:
# Plot current segment as spline:
def lines_from_points(points):
    """Given an array of points, make a line set"""
    poly = pv.PolyData()
    poly.points = points
    cells = np.full((len(points)-1, 3), 2, dtype=np.int_)
    cells[:, 1] = np.arange(0, len(points)-1, dtype=np.int_)
    cells[:, 2] = np.arange(1, len(points), dtype=np.int_)
    poly.lines = cells
    return poly




## Slider with Callback Function Example:

p = pvqt.BackgroundPlotter() # Use just like you would a pv.Plotter() instance

def on_slider_update_mesh(value):
    curr_i = int(value)
    # point_cloud_fixedSegements = np.vstack((x_fixedSegements, y_fixedSegements, z_fixedSegements)).T
    point_cloud_fixedSegements = np.column_stack((x_fixedSegements[curr_i,:], y_fixedSegements[curr_i,:], z_fixedSegements[curr_i,:]))
    print('point_cloud_fixedSegements: {}\n'.format(np.shape(point_cloud_fixedSegements)))
    curr_animal_point = point_cloud_fixedSegements[-1,:] # Get the last point
    animal_location_sphere = pv.Sphere(radius=0.3)
    animal_direction_arrow = pv.Arrow()
#     actor = p.add_mesh(pyvista.Cube(), show_edges=True)

#     mesh = pv.PolyData(point_cloud_fixedSegements)
#     mesh.plot(point_size=10, style='points')
    pdata = pv.PolyData(point_cloud_fixedSegements) # a mesh
#     pdata['orig_sphere'] = np.arange(np.shape(point_cloud)[0])
    
    last_only_opacity_values = np.zeros_like(x_fixedSegements[curr_i,:])
    last_only_opacity_values[-1] = 1.0
#     print(last_only_opacity_values)
    
#     pdata.point_data['pho_fade_values'] = np.arange(np.shape(point_cloud_fixedSegements)[0])
    pdata.point_data['pho_fade_values'] = last_only_opacity_values

    
    
    # create many spheres from the point cloud
    pc = pdata.glyph(scale=False, geom=animal_location_sphere)

    p.add_mesh(pc, color="grey", opacity='linear', scalars='pho_fade_values', nan_opacity=0.0)
#     p.add_mesh(animal_location_sphere, color="grey", ambient=0.6, opacity=0.5, show_edges=False)
    
    # Create spline with 1000 interpolation points
    spline = pv.Spline(point_cloud_fixedSegements)
#     # add scalars to spline and plot it
    spline["scalars"] = np.arange(spline.n_points)
    tube = spline.tube(radius=0.1)
#     tube.plot(smooth_shading=True)
    p.add_mesh(tube, name='tube')

#     sphere = pv.Sphere(phi_resolution=res, theta_resolution=res)
#     p.add_mesh(sphere, name='sphere', show_edges=True)
    return

p.add_slider_widget(on_slider_update_mesh, [0, 100], title='Trajectory Timestep')
p.show()


point_cloud_fixedSegements: (30, 3)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1.]


In [ ]:
# scales = {
#     'x': bqplot.scales.LinearScale(min=10**-3, max=10**3),
#     'y': bqplot.scales.LinearScale(min=-3, max=3),
#     'z': bqplot.scales.LinearScale(min=-3, max=3),
# }
# color_scale = bqplot.scales.ColorScale(min=0, max=3, colors=["#f00", "#0f0", "#00f"])

In [ ]:
x, y, z, u, v, w = np.random.random((6, 1000))*2-1
selected = np.random.randint(0, 1000, 100)
ipv.figure()
quiver = ipv.quiver(x, y, z, u, v, w, size=5, size_selected=8, selected=selected)

from ipywidgets import FloatSlider, ColorPicker, VBox, jslink
size = FloatSlider(min=0, max=30, step=0.1)
size_selected = FloatSlider(min=0, max=30, step=0.1)
color = ColorPicker()
color_selected = ColorPicker()
jslink((quiver, 'size'), (size, 'value'))
jslink((quiver, 'size_selected'), (size_selected, 'value'))
jslink((quiver, 'color'), (color, 'value'))
jslink((quiver, 'color_selected'), (color_selected, 'value'))
VBox([ipv.gcc(), size, size_selected, color, color_selected])